In [56]:
import pandas as pd

result = pd.read_csv('tweet_covid_result_bert.csv')
covid = pd.read_csv('./project-data/tweet_covid_bert.tsv')

In [57]:
result

,Id,Predicted
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
17453,17453,0
17454,17454,0
17455,17455,0
17456,17456,0


In [58]:
covid

,sentence
0,according new york times warner bros wanted de...
1,hurricane hanna made landfall texas storm hitt...
2,monkeys loose india stolen coronavirus blood s...
3,let play blind work fighting islamic religion ...
4,trump felt comfortable comfortable said one su...
...,...
17453,wonder many lives could saved trump focused fi...
17454,front page th march first line reads britain p...
17455,trump completed racism trifecta three minute s...
17456,photographs today division proceedings pandemi...


In [59]:
new_data = pd.DataFrame()

In [60]:
new_data['sentence'] = covid['sentence'].copy()
new_data['Predicted'] = result['Predicted'].copy()

In [61]:
new_data_rumor = new_data[new_data['Predicted'] == 1]
new_data_nonrumor = new_data[new_data['Predicted'] == 0]

In [62]:
# pd.to_csv(new_data,'./tweet_covid_bert.csv')

In [73]:
def get_LDA(new_data = new_data):
    def lemmatize_stemming(text):
        return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    def preprocess(text):
        result = []
        for token in gensim.utils.simple_preprocess(text):
            if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
                result.append(lemmatize_stemming(token))
        return result
    import gensim
    from nltk.stem import WordNetLemmatizer, SnowballStemmer
    import numpy as np
    np.random.seed(2018)
    import nltk
    nltk.download('wordnet')
    stemmer = SnowballStemmer('english')
    processed_data = new_data['sentence'].map(preprocess)
    dictionary = gensim.corpora.Dictionary(processed_data)
    count = 0
    # for k, v in dictionary.iteritems():
    #     print(k, v)
    #     count += 1
    #     if count > 10:
    #         break
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_data]
    from gensim import corpora, models

    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]

    print('###################LDA BOW######################')
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=None)
    for idx, topic in lda_model.print_topics(-1):
        print('Topic: {} \nWords: {}'.format(idx, topic))
    print('###################LDA TFIDF######################')
    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
    for idx, topic in lda_model_tfidf.print_topics(-1):
        print('Topic: {} Word: {}'.format(idx, topic))

In [74]:
get_LDA(new_data_rumor)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yujia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


###################LDA BOW######################
Topic: 0 
Words: 0.072*"coronavirus" + 0.023*"covid" + 0.023*"trump" + 0.020*"peopl" + 0.010*"think" + 0.010*"test" + 0.010*"elect" + 0.010*"like" + 0.010*"corona" + 0.009*"american"
Topic: 1 
Words: 0.050*"covid" + 0.039*"trump" + 0.038*"coronavirus" + 0.017*"american" + 0.015*"death" + 0.015*"peopl" + 0.014*"like" + 0.012*"know" + 0.009*"die" + 0.009*"say"
Topic: 2 
Words: 0.047*"covid" + 0.036*"coronavirus" + 0.023*"school" + 0.020*"trump" + 0.019*"want" + 0.018*"like" + 0.018*"peopl" + 0.011*"children" + 0.010*"death" + 0.009*"think"
Topic: 3 
Words: 0.041*"covid" + 0.041*"coronavirus" + 0.031*"death" + 0.030*"american" + 0.023*"trump" + 0.017*"peopl" + 0.016*"rat" + 0.014*"die" + 0.013*"kill" + 0.012*"case"
Topic: 4 
Words: 0.128*"coronavirus" + 0.057*"elect" + 0.034*"trump" + 0.024*"general" + 0.021*"worri" + 0.016*"like" + 0.015*"covid" + 0.015*"peopl" + 0.011*"kill" + 0.010*"think"
Topic: 5 
Words: 0.066*"covid" + 0.032*"trump" +

In [75]:
get_LDA(new_data_nonrumor)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yujia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


###################LDA BOW######################
Topic: 0 
Words: 0.020*"test" + 0.016*"peopl" + 0.016*"trump" + 0.015*"death" + 0.008*"mask" + 0.008*"like" + 0.007*"know" + 0.006*"virus" + 0.006*"american" + 0.006*"presid"
Topic: 1 
Words: 0.017*"peopl" + 0.017*"case" + 0.013*"trump" + 0.011*"death" + 0.007*"test" + 0.007*"need" + 0.007*"know" + 0.006*"like" + 0.006*"state" + 0.006*"countri"
Topic: 2 
Words: 0.012*"trump" + 0.011*"peopl" + 0.010*"death" + 0.009*"pandem" + 0.009*"american" + 0.009*"test" + 0.007*"know" + 0.006*"care" + 0.006*"countri" + 0.005*"home"
Topic: 3 
Words: 0.017*"trump" + 0.015*"death" + 0.012*"peopl" + 0.011*"american" + 0.008*"case" + 0.007*"like" + 0.006*"die" + 0.006*"dead" + 0.006*"number" + 0.006*"pandem"
Topic: 4 
Words: 0.021*"trump" + 0.014*"death" + 0.014*"american" + 0.012*"peopl" + 0.009*"state" + 0.009*"case" + 0.008*"presid" + 0.008*"countri" + 0.008*"virus" + 0.006*"know"
Topic: 5 
Words: 0.021*"trump" + 0.013*"peopl" + 0.012*"death" + 0.008*"p